In [ ]:
# 必要なライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc
import warnings
warnings.filterwarnings('ignore')

# 日本語表示の設定
plt.rcParams['font.sans-serif'] = ['Hiragino Sans', 'Yu Gothic', 'Meirio', 'Takao', 'IPAexGothic', 'IPAPGothic', 'VL PGothic', 'Noto Sans CJK JP']
plt.rcParams['axes.unicode_minus'] = False

print("ライブラリのインポートが完了しました")


In [ ]:
# Pima Indians Diabetes DatasetをGitHubから読み込み
url = "https://raw.githubusercontent.com/npradaschnor/Pima-Indians-Diabetes-Dataset/master/diabetes.csv"
df = pd.read_csv(url)

# データの確認
print("データの形状:", df.shape)
print("\n列名:")
print(df.columns.tolist())
print("\n最初の10行:")
df.head(10)


In [ ]:
# データの基本統計量
print("基本統計量:")
print(df.describe())

# 糖尿病の有無の分布
print("\n糖尿病の分布:")
print(df['Outcome'].value_counts())
print(f"\n糖尿病の割合: {df['Outcome'].mean()*100:.1f}%")

# データ型の確認
print("\nデータ型:")
print(df.dtypes)


In [ ]:
# 生理学的にありえない0の値を確認
print("各列の0の数:")
print((df == 0).sum())

# 0が欠損値として扱われるべき列
zero_as_missing = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

print("\n\n生理学的にありえない0の割合:")
for col in zero_as_missing:
    zero_count = (df[col] == 0).sum()
    zero_percent = zero_count / len(df) * 100
    print(f"{col}: {zero_count}件 ({zero_percent:.1f}%)")

# 0を含む行の例を表示
print("\n\n0を含むデータの例:")
df[df[zero_as_missing].eq(0).any(axis=1)].head(10)


In [ ]:
# データの可視化（0を含む生データ）
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.ravel()

# 各特徴量の分布を糖尿病の有無で比較
features = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 
            'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']

for i, feature in enumerate(features):
    df[df['Outcome']==0][feature].hist(bins=30, alpha=0.5, label='健常者', ax=axes[i], color='blue')
    df[df['Outcome']==1][feature].hist(bins=30, alpha=0.5, label='糖尿病', ax=axes[i], color='red')
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel('人数')
    axes[i].legend()
    axes[i].set_title(f'{feature}の分布')
    
    # 0の値に赤い線を追加（問題のある値を強調）
    if feature in zero_as_missing:
        axes[i].axvline(x=0, color='red', linestyle='--', alpha=0.7, label='欠損値(0)')

# 最後の軸を削除
fig.delaxes(axes[8])

plt.tight_layout()
plt.show()

print("注意: Glucose, BloodPressure, SkinThickness, Insulin, BMIの0は生理学的にありえない値です。")


In [ ]:
# 相関行列の可視化（生データ）
plt.figure(figsize=(10, 8))
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('特徴量間の相関係数（欠損値を含む生データ）', fontsize=16)
plt.show()

# 糖尿病との相関が高い特徴量
print("糖尿病(Outcome)との相関係数:")
print(correlation_matrix['Outcome'].sort_values(ascending=False))


In [ ]:
# データのコピーを作成（元データを保持）
df_clean = df.copy()

# Step 1: 0を欠損値（NaN）に変換
for col in zero_as_missing:
    df_clean.loc[df_clean[col] == 0, col] = np.nan

# 欠損値の状況を確認
print("欠損値の数:")
print(df_clean.isnull().sum())
print(f"\n全体の欠損値の割合: {df_clean.isnull().sum().sum() / (len(df_clean) * len(df_clean.columns)) * 100:.1f}%")

# 欠損値の視覚化
plt.figure(figsize=(12, 6))
sns.heatmap(df_clean.isnull(), cbar=True, yticklabels=False, cmap='viridis')
plt.title('欠損値のパターン（黄色が欠損値）')
plt.tight_layout()
plt.show()


In [ ]:
# 異なる欠損値処理方法の比較

# 方法1: 欠損値を含む行を削除
df_dropna = df_clean.dropna()
print(f"方法1 - 行削除: {len(df_dropna)}行 (元の{len(df_dropna)/len(df)*100:.1f}%)")

# 方法2: 平均値で補完
df_mean = df_clean.copy()
for col in zero_as_missing:
    mean_val = df_clean[col].mean()
    df_mean[col].fillna(mean_val, inplace=True)
print(f"方法2 - 平均値補完: {len(df_mean)}行 (データ保持)")

# 方法3: 中央値で補完
df_median = df_clean.copy()
for col in zero_as_missing:
    median_val = df_clean[col].median()
    df_median[col].fillna(median_val, inplace=True)
print(f"方法3 - 中央値補完: {len(df_median)}行 (データ保持)")

# 方法4: 条件付き補完（糖尿病の有無で分けて補完）
df_conditional = df_clean.copy()
for col in zero_as_missing:
    # 健常者の中央値
    median_healthy = df_clean[df_clean['Outcome'] == 0][col].median()
    # 糖尿病患者の中央値
    median_diabetic = df_clean[df_clean['Outcome'] == 1][col].median()
    
    # 条件付きで補完
    df_conditional.loc[(df_conditional[col].isnull()) & (df_conditional['Outcome'] == 0), col] = median_healthy
    df_conditional.loc[(df_conditional[col].isnull()) & (df_conditional['Outcome'] == 1), col] = median_diabetic

print(f"方法4 - 条件付き補完: {len(df_conditional)}行 (データ保持)")


In [ ]:
# 欠損値処理前後の分布比較（Glucoseを例に）
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 元のデータ（0を含む）
axes[0,0].hist(df[df['Outcome']==0]['Glucose'], bins=30, alpha=0.5, label='健常者', color='blue')
axes[0,0].hist(df[df['Outcome']==1]['Glucose'], bins=30, alpha=0.5, label='糖尿病', color='red')
axes[0,0].axvline(x=0, color='red', linestyle='--', alpha=0.7)
axes[0,0].set_title('元データ（0を含む）')
axes[0,0].set_xlabel('Glucose')
axes[0,0].legend()

# 行削除後
axes[0,1].hist(df_dropna[df_dropna['Outcome']==0]['Glucose'], bins=30, alpha=0.5, label='健常者', color='blue')
axes[0,1].hist(df_dropna[df_dropna['Outcome']==1]['Glucose'], bins=30, alpha=0.5, label='糖尿病', color='red')
axes[0,1].set_title(f'行削除後（n={len(df_dropna)}）')
axes[0,1].set_xlabel('Glucose')
axes[0,1].legend()

# 中央値補完後
axes[1,0].hist(df_median[df_median['Outcome']==0]['Glucose'], bins=30, alpha=0.5, label='健常者', color='blue')
axes[1,0].hist(df_median[df_median['Outcome']==1]['Glucose'], bins=30, alpha=0.5, label='糖尿病', color='red')
axes[1,0].set_title('中央値補完後')
axes[1,0].set_xlabel('Glucose')
axes[1,0].legend()

# 条件付き補完後
axes[1,1].hist(df_conditional[df_conditional['Outcome']==0]['Glucose'], bins=30, alpha=0.5, label='健常者', color='blue')
axes[1,1].hist(df_conditional[df_conditional['Outcome']==1]['Glucose'], bins=30, alpha=0.5, label='糖尿病', color='red')
axes[1,1].set_title('条件付き補完後')
axes[1,1].set_xlabel('Glucose')
axes[1,1].legend()

plt.tight_layout()
plt.show()

print("考察: 条件付き補完は、健常者と糖尿病患者の分布の違いを保持しながら欠損値を処理できています。")


In [ ]:
# 条件付き補完データを採用
df_processed = df_conditional.copy()

print("処理後のデータの確認:")
print(f"データ形状: {df_processed.shape}")
print(f"欠損値の数: {df_processed.isnull().sum().sum()}")

# 処理後の基本統計量
print("\n処理後の基本統計量:")
print(df_processed.describe())


In [ ]:
# 特徴量とターゲットの分離（クリーニング済みデータを使用）
X = df_processed.drop('Outcome', axis=1)  # 説明変数
y = df_processed['Outcome']  # 目的変数

print("特徴量の形状:", X.shape)
print("特徴量の列:", X.columns.tolist())

# 訓練データとテストデータの分割（8:2の割合）
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n訓練データ: {X_train.shape[0]}件")
print(f"テストデータ: {X_test.shape[0]}件")

# クラスの分布を確認（偏りがないか）
print(f"\n訓練データの糖尿病割合: {y_train.mean()*100:.1f}%")
print(f"テストデータの糖尿病割合: {y_test.mean()*100:.1f}%")


In [ ]:
# 特徴量の標準化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 標準化前後の比較
print("標準化前の統計量（訓練データの一部）:")
print(X_train.describe().iloc[:2, :3])
print("\n標準化後の統計量:")
print(pd.DataFrame(X_train_scaled, columns=X_train.columns).describe().iloc[:2, :3])


In [ ]:
# Logistic Regressionモデルの作成と訓練
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train_scaled, y_train)

# 予測
lr_pred = lr_model.predict(X_test_scaled)
lr_pred_proba = lr_model.predict_proba(X_test_scaled)[:, 1]

# 精度の評価
lr_accuracy = accuracy_score(y_test, lr_pred)
print(f"Logistic Regressionの精度: {lr_accuracy:.3f}")

# 混同行列
print("\n混同行列:")
cm_lr = confusion_matrix(y_test, lr_pred)
print(cm_lr)

# 分類レポート
print("\n分類レポート:")
print(classification_report(y_test, lr_pred, target_names=['健常者', '糖尿病']))


In [ ]:
# 特徴量の重要度（係数）の可視化
coefficients = pd.DataFrame({
    '特徴量': X.columns,
    '係数': lr_model.coef_[0]
})
coefficients = coefficients.sort_values('係数', key=abs, ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(coefficients['特徴量'], coefficients['係数'])
plt.xlabel('係数')
plt.title('Logistic Regressionの係数（特徴量の重要度）')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print("係数の解釈:")
print("- 正の係数: その特徴量が大きいほど糖尿病リスクが高い")
print("- 負の係数: その特徴量が大きいほど糖尿病リスクが低い")
print(f"\n最も重要な特徴量: {coefficients.iloc[0]['特徴量']}")


In [ ]:
# Decision Treeモデルの作成と訓練
# 注：Decision Treeは標準化が不要なので、元のデータを使用
dt_model = DecisionTreeClassifier(
    max_depth=4,  # 木の深さを制限（過学習を防ぐ）
    min_samples_split=20,  # 分割に必要な最小サンプル数
    random_state=42
)
dt_model.fit(X_train, y_train)

# 予測
dt_pred = dt_model.predict(X_test)
dt_pred_proba = dt_model.predict_proba(X_test)[:, 1]

# 精度の評価
dt_accuracy = accuracy_score(y_test, dt_pred)
print(f"Decision Treeの精度: {dt_accuracy:.3f}")

# 混同行列
print("\n混同行列:")
cm_dt = confusion_matrix(y_test, dt_pred)
print(cm_dt)

# 分類レポート
print("\n分類レポート:")
print(classification_report(y_test, dt_pred, target_names=['健常者', '糖尿病']))


In [ ]:
# 特徴量の重要度の可視化
feature_importance = pd.DataFrame({
    '特徴量': X.columns,
    '重要度': dt_model.feature_importances_
}).sort_values('重要度', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(feature_importance['特徴量'], feature_importance['重要度'])
plt.xlabel('重要度')
plt.title('Decision Treeの特徴量重要度')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print("特徴量重要度の解釈:")
print("- 値が大きいほど、その特徴量が分類に重要")
print(f"- 最も重要な特徴量: {feature_importance.iloc[0]['特徴量']}")


In [ ]:
# 決定木の可視化（簡易版）
from sklearn.tree import plot_tree

plt.figure(figsize=(20, 10))
plot_tree(dt_model, 
          feature_names=X.columns,
          class_names=['健常者', '糖尿病'],
          filled=True,
          rounded=True,
          fontsize=10)
plt.title('決定木の構造', fontsize=16)
plt.tight_layout()
plt.show()

print("決定木の読み方:")
print("- 各ノードで特徴量の条件を評価")
print("- True（条件を満たす）なら左、False（満たさない）なら右へ")
print("- 色が濃いほど糖尿病の確率が高い")


In [ ]:
# 混同行列の比較
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Logistic Regression
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['健常者', '糖尿病'], yticklabels=['健常者', '糖尿病'])
axes[0].set_title('Logistic Regression')
axes[0].set_ylabel('実際の診断')
axes[0].set_xlabel('予測')

# Decision Tree
sns.heatmap(cm_dt, annot=True, fmt='d', cmap='Blues', ax=axes[1],
            xticklabels=['健常者', '糖尿病'], yticklabels=['健常者', '糖尿病'])
axes[1].set_title('Decision Tree')
axes[1].set_ylabel('実際の診断')
axes[1].set_xlabel('予測')

plt.tight_layout()
plt.show()

# 性能指標の比較
print("モデル性能の比較:")
print(f"Logistic Regression - 精度: {lr_accuracy:.3f}")
print(f"Decision Tree - 精度: {dt_accuracy:.3f}")


In [ ]:
# ROC曲線の比較
plt.figure(figsize=(10, 8))

# Logistic Regression
fpr_lr, tpr_lr, _ = roc_curve(y_test, lr_pred_proba)
auc_lr = auc(fpr_lr, tpr_lr)
plt.plot(fpr_lr, tpr_lr, label=f'Logistic Regression (AUC = {auc_lr:.3f})', linewidth=2)

# Decision Tree
fpr_dt, tpr_dt, _ = roc_curve(y_test, dt_pred_proba)
auc_dt = auc(fpr_dt, tpr_dt)
plt.plot(fpr_dt, tpr_dt, label=f'Decision Tree (AUC = {auc_dt:.3f})', linewidth=2)

# 対角線（ランダム予測）
plt.plot([0, 1], [0, 1], 'k--', label='ランダム予測', linewidth=1)

plt.xlabel('偽陽性率 (1 - 特異度)', fontsize=12)
plt.ylabel('真陽性率 (感度)', fontsize=12)
plt.title('ROC曲線の比較', fontsize=16)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
plt.show()

print("AUC（Area Under Curve）の解釈:")
print("- 1.0に近いほど良いモデル")
print("- 0.5はランダム予測と同じ")
print(f"\nLogistic Regression AUC: {auc_lr:.3f}")
print(f"Decision Tree AUC: {auc_dt:.3f}")


In [ ]:
# 実践例：新しい患者のリスク予測
def predict_diabetes_risk(patient_data, model, scaler=None):
    """
    新しい患者の糖尿病リスクを予測する関数
    """
    # データフレームに変換
    patient_df = pd.DataFrame([patient_data])
    
    # 標準化（Logistic Regressionの場合）
    if scaler is not None:
        patient_scaled = scaler.transform(patient_df)
        prediction_proba = model.predict_proba(patient_scaled)[0]
    else:
        prediction_proba = model.predict_proba(patient_df)[0]
    
    risk_level = prediction_proba[1]
    
    # リスクレベルの判定
    if risk_level < 0.3:
        risk_category = "低リスク"
    elif risk_level < 0.7:
        risk_category = "中リスク"
    else:
        risk_category = "高リスク"
    
    return risk_level, risk_category

# 新しい患者の例（実際のデータセットの特徴量に合わせる）
new_patient = {
    'Pregnancies': 3,
    'Glucose': 115,
    'BloodPressure': 70,
    'SkinThickness': 28,
    'Insulin': 125,
    'BMI': 26.5,
    'DiabetesPedigreeFunction': 0.351,
    'Age': 45
}

print("新しい患者のデータ:")
for key, value in new_patient.items():
    print(f"  {key}: {value}")

# 両モデルで予測
lr_risk, lr_category = predict_diabetes_risk(new_patient, lr_model, scaler)
dt_risk, dt_category = predict_diabetes_risk(new_patient, dt_model)

print(f"\n予測結果:")
print(f"Logistic Regression: {lr_risk:.1%} ({lr_category})")
print(f"Decision Tree: {dt_risk:.1%} ({dt_category})")

print("\n推奨アクション:")
if lr_category == "高リスク" or dt_category == "高リスク":
    print("⚠️ 精密検査を推奨")
elif lr_category == "中リスク" or dt_category == "中リスク":
    print("📋 生活習慣の改善と定期的なフォローアップを推奨")
else:
    print("✅ 現時点でのリスクは低いが、定期健診を継続")
